# **Neural Language Modeling**


1.   **Library** (nltk, torch)
2.   **Result** (Word prediction using Neural Networks)




In [1]:
import io
import re
from nltk import FreqDist
from nltk.util import ngrams
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

Downlaod data

In [2]:
!gdown --id 1qCVYpb67RuzUbyrJ3w-ohtCf-tzZKTal
!gdown --id 1dW5SkCYIFbXmNe3xKv4EhrLPDPlXyIDy

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1qCVYpb67RuzUbyrJ3w-ohtCf-tzZKTal
To: /content/train.txt
100% 9.87M/9.87M [00:00<00:00, 30.9MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1dW5SkCYIFbXmNe3xKv4EhrLPDPlXyIDy
To: /content/test.txt
100% 5.80k/5.80k [00:00<00:00, 23.4MB/s]


Preprocess

> Ignoring

*   extra consecutive spaces
*   spaces at the beginning and end of lines

In [3]:
def file_clear(in_f, out_f):
    inputFile = io.open(in_f, mode="r", encoding="utf-8")
    outputFile = io.open(out_f, mode="w", encoding="utf-8")

    updated=[]
    Lines = inputFile.readlines()
    for i, line in enumerate(Lines):
        newline= re.sub(' +', ' ', line.strip())
        if len(line.split()) >2:
            updated.append(newline+"\n")
        else:
            print(line,i)
    outputFile.writelines(updated)
    print( len(Lines), len(updated))
    inputFile.__exit__()
    outputFile.__exit__()
#updated

file_clear('train.txt', 'cleaned_train.txt')
file_clear('test.txt', 'cleaned_test.txt')

بسیارخوری وکامرانی
 2438
هنگام مکافات
 31304
حضرت ستارخان
 57052
سلام چطوری
 67800
گلبنان ممتحن
 74076
همدست وهمداستان
 78314
ای نازنین
 96936
قصر چلستون
 101252
بسیار سخن
 107340
اندر آذربایجان
 108670
نپذیرند یکی
 118562
بیشتر کن
 140401
بیچاره         رعیت
 151833
اندرگلستان ما
 152975
ناگاه صبا
 159903
پرشرر کن
 167759
همدرد ندارد
 175501
برملت خوبشتن
 184457
188894 188876
108 108


Q. Complete the following expressions using the neural language modeling(trigram)


1.   چون مشک سیه بود مرا هر دو بنا
2.   گر خورد سوگند هم آن
3.   زانک نفس آشفته تر گردد از
4.   ازین زشت تر در جهان رنگ
5.   دوست در خانه و ما گرد
6.   شب است و شمع و شراب و



In [4]:
#Remove Big Prev Model
lm = None

#Get Words
def compute_freq(inputfile, nu):
    textfile = io.open(inputfile, mode="r", encoding="utf-8")
    Lines = textfile.readlines()
    textfile.__exit__()
    ngramfdist = FreqDist()

    for i,line in enumerate(Lines):
        tokens = line.strip().split(' ')
        if len(tokens)>nu:
            # ngram_ = ngrams(tokens, nu)
            # print(i+1," ",line)
            ngram_ = ngrams(tokens, nu)
            ngramfdist.update(ngram_)
    return ngramfdist

words = compute_freq("cleaned_train.txt",1)
#words_dict = collections.Counter(ngrams)
words = [i[0] for i in words]
word2int = dict(enumerate(words))
word2int = {word: ind for ind, word in word2int.items()}
int2word = {value: key for key, value in word2int.items()}
int2word.keys()

vocab_size= len(int2word)

#Get Trigrams
def generateSeq(inputfile, nu):
    textfile = io.open(inputfile, mode="r", encoding="utf-8")
    Lines = textfile.readlines()
    textfile.__exit__()
    sequences={}
    seqIndex=0
    for i,line in enumerate(Lines):
        tokens = line.strip().split(' ')
        if len(tokens)>nu:
            ngrams_ = ngrams(tokens, nu)
            for tuple_ in ngrams_:
                t = list(tuple_)
                sequences[seqIndex]= ([word2int[i] for i in t[:-1]], word2int[t[-1]] )
                seqIndex = seqIndex+1
    return sequences

sequences_dict = generateSeq("cleaned_train.txt",3)

In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 128

class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, sequence_dict):
        'Initialization'
        self.data_dict = sequence_dict
        # self.labels = labels
        # self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data_dict)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        # ID = self.list_IDs[index]

        # # Load data and get label
        # X = torch.load('data/' + ID + '.pt')
        # y = self.labels[ID]
        x,y = self.data_dict[index]
        return tuple(x), y

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        #embeds = self.embeddings(inputs).view((1, -1))
        embeds= torch.cat((self.embeddings(inputs[0]),self.embeddings(inputs[1])),1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# Parameters
params = {'batch_size': 1024*8,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 100

# Generators
training_set = Dataset(sequences_dict)
training_generator = torch.utils.data.DataLoader(training_set, **params)


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(vocab_size, EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(max_epochs):
    total_loss = 0
    for context, target in training_generator:

        context_idxs = [context[0].to(device), context[1].to(device)]
        model.zero_grad()

        log_probs = model(context_idxs)

        loss = loss_function(log_probs, target.to(device))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(epoch, " ",total_loss)
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"

0   970.9450263977051
1   835.6389660835266
2   812.0403671264648
3   792.7543659210205
4   774.0541157722473
5   754.7730650901794
6   734.35977602005
7   713.5832123756409
8   694.047770023346
9   677.6942548751831
10   665.0354347229004
11   655.4392976760864
12   647.4949808120728
13   640.7144150733948
14   634.7122550010681
15   629.3551740646362
16   624.405110836029
17   619.8325300216675
18   615.5947489738464
19   611.6133108139038
20   607.8593854904175
21   604.3041806221008
22   600.8442893028259
23   597.6631760597229
24   594.476667881012
25   591.4806203842163
26   588.6163301467896
27   585.8338694572449
28   583.0346217155457
29   580.4711384773254
30   577.9402618408203
31   575.4376316070557
32   572.9972620010376
33   570.6916518211365
34   568.4208574295044
35   566.1785836219788
36   563.978741645813
37   561.8296270370483
38   559.7760863304138
39   557.705319404602
40   555.7080602645874
41   553.7185559272766
42   551.85515832901
43   550.0347504615784
44   54

In [6]:
sents_to_comp=[ ("چون مشک سیه بود مرا هر دو", 2),
                ("گر خورد سوگند هم آن", 1),
                ("زانک نفس آشفته تر گردد از",1),
                ("ازین زشت تر در جهان رنگ",1),
                ("دوست در خانه و ما گرد",2),
                ("شب است و شمع و شراب و",1)
]

model.eval()
for sent, i2 in  sents_to_comp:
    for i in range(i2):
        sent_words= sent.strip().split()
        context = [ torch.tensor([word2int[sent_words[-2]]]).to(device) , torch.tensor([word2int[sent_words[-1]]]).to(device)]
        nextWordPreds = model(context)
        nextWordID = np.argmax(nextWordPreds.detach().cpu()).item()
        nextWord = int2word[nextWordID]
        sent = sent + " "+ nextWord
    print(sent)

چون مشک سیه بود مرا هر دو را به
گر خورد سوگند هم آن را
زانک نفس آشفته تر گردد از آن
ازین زشت تر در جهان رنگ و
دوست در خانه و ما گرد او را
شب است و شمع و شراب و کباب
